### Patch dataquality to log to a local store for testing

In [ ]:
import os 
from uuid import uuid4
os.environ['GALILEO_MINIO_ACCESS_KEY'] = "test"
os.environ['GALILEO_MINIO_SECRET_KEY'] = "test"
os.environ['GALILEO_MINIO_URL'] = "test"

import dataquality
from dataquality import config
from dataquality.clients import object_store
from dataquality.loggers.jsonl_logger import JsonlLogger

proj = uuid4()
run = uuid4()
config.current_project_id = proj
config.current_run_id = run

LOCATION = (
    f"{JsonlLogger.LOG_FILE_DIR}/{config.current_project_id}"
    f"/{config.current_run_id}"
)

TEST_STORE_DIR = "TEST_STORE"
TEST_PATH = f"{LOCATION}/{TEST_STORE_DIR}"


def patch_object_upload(df, object_name: str) -> None:
    """
    A patch for the object_store.create_project_run_object_from_df so we don't have to
    talk to minio for testing
    """
    # separate folder per split (test, train, val) and data type (emb, prob, data)
    split, epoch, data_type, file_name = object_name.split("/")[-4:]
    export_path = f"{TEST_PATH}/{split}/{data_type}"
    export_loc = f"{export_path}/{file_name}"

    if not os.path.isdir(export_path):
        os.makedirs(export_path)
    df.export_arrow(export_loc)
    
object_store.create_project_run_object_from_df = patch_object_upload

In [ ]:
from dataquality.core.integrations.config import GalileoDataConfig, GalileoModelConfig
from random import random


num_records=4

dataquality.log_batch_input_data(GalileoDataConfig(
    text=["test","test2","test3","test4"],
    labels=["false","true","false","true"],
    ids=[1,2,3,4],
    split="train"
))


emb = [[random() for _ in range(20)] for _ in range(num_records)]
probs = [[random() for _ in range(4)] for _ in range(num_records)]
epoch = 0
ids = list(range(num_records))

model_config = GalileoModelConfig(
    emb=emb, probs=probs, split="train", epoch=epoch, ids=ids
)
dataquality.log_model_outputs(model_config)

In [ ]:
!ls -lsa .galileo/logs/{proj}/{run}

In [ ]:
dataquality.core.finish._upload()

In [ ]:
!tree .galileo/logs/{proj}/{run}